In [1]:
import numpy as np
from scipy.stats import entropy
import json
import collections
import itertools

import powerlaw
from jointpdf.jointpdf import JointProbabilityMatrix
from jointpdf.jointpdf import FullNestedArrayOfProbabilities

from probability_distributions import JointProbabilityMatrixExtended
import probability_distributions
from probability_distributions import ProbabilityArray
from simulate import find_mean_std_mse
import nudge

In [2]:
pdf = JointProbabilityMatrix(2, 5, 'random')
#pdf.append_variables_with_target_mi(1, 0.5)
#pdf.append_synergistic_variables()

import numpy as np

probability_array_tryout = FullNestedArrayOfProbabilities(
    np.array(
      [
        [
          [
            [0.2, 0.1]          
          ],
          [
            [0.05, 0.05]
          ]
        ],
        [
          [
            [0.3, 0.05] 
          ],
          [
            [0.15, 0.1] 
          ]
        ],
      ]
    )
) 

In [3]:
NUDGE_SIZE = 0.01
pdf = JointProbabilityMatrix(1, 6, 'random')
#print(pdf.joint_probabilities.joint_probabilities)
pdf.append_variables_with_target_mi(1, 0.1)
#print(pdf.mutual_information([0], [1]))

joint_old = pdf.joint_probabilities.joint_probabilities
probability_array_old = ProbabilityArray(joint_old)
marginal_variable_old = probability_array_old.marginalize(set([0]))
marginal_function_old = probability_array_old.marginalize(set([1]))
conditional_joint_old, marginal_labels_old, conditional_labels_old = (
    probability_array_old.find_conditional(set([1]), set([0]))
)
marginal_variable_nudged, nudged_states = nudge.nudge(marginal_variable_old, NUDGE_SIZE)
joint_new = ProbabilityArray(probability_distributions.compute_joint(
    marginal_variable_nudged, conditional_joint_old, conditional_labels_old
))

#This takes the KL-divergence between the new and old function variable
marginal_function_new = joint_new.marginalize(set([1]))  

kl_variable = entropy(marginal_variable_old, marginal_variable_nudged)
kl_function = entropy(marginal_function_old, marginal_function_new) 
print("KL-divergence old and new function distribution: {}".format(kl_variable))
print("KL-divergence old and new function distribution: {}".format(kl_function))

KL-divergence old and new function distribution: 0.000777099901024
KL-divergence old and new function distribution: 1.93103500031e-05


In [4]:
#this method does ot work, the joint does not sum to 1!!!
def update_joint_independent_marginals(joint, label_marginal, marginal, 
                                       update_states, other_marginal):
    """update a joint distribution only for certain states of a marginal
    assuming the merginals are independent for those states
    
    Parameters:
    ----------
    joint: numpy array
    label_marginal: integer
    marginal: a numpy array
    states: a numpy array
    other_marginal: a numpy array
    
    Returns: The updated joint distribution

    """
    updated_joint = np.copy(joint)
    states_input_variables = [range(states) for states in joint.shape]
    states_input_variables[label_marginal] = list(update_states)
    
    for state in itertools.product(*states_input_variables):
        state_other_input_variables = tuple([index for count, index in enumerate(state)
                                             if count != label_marginal])
        updated_joint[state] = (other_marginal[state_other_input_variables] *
                                marginal_variable_nudged[state[label_marginal]])

    return updated_joint



In [ ]:
import numpy as np

tryout = np.arange(2**10).reshape([2]*10)
#print(tryout)
a = [1, 3, 4, 5, 9]
total_variables = 10
b = np.moveaxis(tryout, a, range(total_variables-len(a), total_variables, 1))
c = np.array([[1,10],[100,1000]])
b = b*c
#tryout = np.moveaxis(b, range(total_variables-len(a), total_variables, 1), a)
print(tryout)


#tryout = b*c
#np.moveaxis(tryout, range(total_variables-len(a), total_variables, 1), a)
